In [2]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

import json
import types
from collections import OrderedDict
from pprint import pprint

import torch
import torch.nn as nn
from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [3]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10"
SETTINGS = [
    {
        "normalization_mean": [-0.731, -0.311, 0.544],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.219, -0.247, -0.174],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.721, 0.778, -0.601],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.749, -0.448, -0.776],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.311, -0.857, 0.897],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.205, 0.181, -0.549],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.804, 0.685, 0.01],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.6, -0.734, 0.089],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.511, -0.221, 0.837],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.853, -0.615, -0.149],
        "normalization_std": None,
    },
]


In [4]:
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    print(label)


model__mean_-0.731_-0.311_0.544
model__mean_0.219_-0.247_-0.174
model__mean_-0.721_0.778_-0.601
model__mean_0.749_-0.448_-0.776
model__mean_-0.311_-0.857_0.897
model__mean_0.205_0.181_-0.549
model__mean_-0.804_0.685_0.01
model__mean_0.6_-0.734_0.089
model__mean_-0.511_-0.221_0.837
model__mean_0.853_-0.615_-0.149


In [5]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [5]:
model_name__tr_acc = OrderedDict()

# model performance in TR
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"
    args.experiment = 'diff-norm-value'
    args.dataset = 'CIFAR10'
    args.batch_size = 1000
    args.num_workers = 8
    args.normalization_mean = normalization_mean
    args.normalization_std = normalization_std
    args.extra_preprocessing_type = ""
    train_loader, test_loader = get_new_dataloader(args=args)

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__tr_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }


/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_-0.631_-0.648_0.624/final.pt
Custom_normalization: Normalize(mean=[-0.631, -0.648, 0.624], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:28<00:00,  1.76it/s]


0.99998
0.9425
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_-0.631_-0.97_-0.058/final.pt
Custom_normalization: Normalize(mean=[-0.631, -0.97, -0.058], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


0.99998
0.9433
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_-0.668_-0.004_0.165/final.pt
Custom_normalization: Normalize(mean=[-0.668, -0.004, 0.165], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


...

0.99996
0.9409
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_0.78_0.026_-0.51/final.pt
Custom_normalization: Normalize(mean=[0.78, 0.026, -0.51], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


1.0
0.9398
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_0.847_-0.447_0.64/final.pt
Custom_normalization: Normalize(mean=[0.847, -0.447, 0.64], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


1.0
0.9415


In [6]:
pprint(model_name__tr_acc)

# Serializing json
json_object = json.dumps(model_name__tr_acc, indent=2)
 
# Writing to sample.json
with open("tr_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model__mean_-0.631_-0.648_0.624',
              {'test_acc': 0.9425, 'train_acc': 0.99998}),
             ('model__mean_-0.631_-0.97_-0.058',
              {'test_acc': 0.9433, 'train_acc': 0.99998}),
             ('model__mean_-0.668_-0.004_0.165',
              {'test_acc': 0.9417, 'train_acc': 0.99998}),
             ('model__mean_0.26_0.855_-0.536',
              {'test_acc': 0.9428, 'train_acc': 0.99996}),
             ('model__mean_0.365_-0.892_-0.559',
              {'test_acc': 0.9436, 'train_acc': 0.99994}),
             ('model__mean_0.456_0.837_0.251',
              {'test_acc': 0.9447, 'train_acc': 0.99998}),
             ('model__mean_0.598_0.036_-0.537',
              {'test_acc': 0.942, 'train_acc': 0.99998}),
             ('model__mean_0.648_-0.572_0.483',
              {'test_acc': 0.9409, 'train_acc': 0.99996}),
             ('model__mean_0.78_0.026_-0.51',
              {'test_acc': 0.9398, 'train_acc': 1.0}),
             ('model__mean_0.847_-0.447_0.

In [6]:
model_name__te_acc = OrderedDict()

# model performance in FE
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=True,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__te_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_-0.731_-0.311_0.544/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:35<00:00,  1.39it/s]


0.10014
0.1001
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.219_-0.247_-0.174/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


0.35806
0.3572
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_-0.721_0.778_-0.601/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.13936
0.1433
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_-0.511_-0.221_0.837/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.61it/s]


0.10034
0.1001
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.853_-0.615_-0.149/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


0.09476
0.0913


In [7]:
pprint(model_name__te_acc)

# Serializing json
json_object = json.dumps(model_name__te_acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model__mean_-0.731_-0.311_0.544',
              {'test_acc': 0.1001, 'train_acc': 0.10014}),
             ('model__mean_0.219_-0.247_-0.174',
              {'test_acc': 0.3572, 'train_acc': 0.35806}),
             ('model__mean_-0.721_0.778_-0.601',
              {'test_acc': 0.1426, 'train_acc': 0.14054}),
             ('model__mean_0.749_-0.448_-0.776',
              {'test_acc': 0.1063, 'train_acc': 0.10594}),
             ('model__mean_-0.311_-0.857_0.897',
              {'test_acc': 0.1449, 'train_acc': 0.1436}),
             ('model__mean_0.205_0.181_-0.549',
              {'test_acc': 0.1802, 'train_acc': 0.1805}),
             ('model__mean_-0.804_0.685_0.01',
              {'test_acc': 0.1432, 'train_acc': 0.13994}),
             ('model__mean_0.6_-0.734_0.089',
              {'test_acc': 0.1433, 'train_acc': 0.13936}),
             ('model__mean_-0.511_-0.221_0.837',
              {'test_acc': 0.1001, 'train_acc': 0.10034}),
             ('model__mean_0.853_-0.